In [1]:
import tensorflow.keras as keras
import tensorflow.compat.v2 as tf
from utils.read_mnist import read_mnist
import lenet

In [2]:
train_x, train_y, test_x, test_y = read_mnist(one_hot=True, standard=True)

In [3]:
model = lenet.model(regular=0.00001)

In [4]:
tensorboardlog = r"trainlogs\\"
trainlogdir = tensorboardlog
scalarlogdir = tensorboardlog
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=trainlogdir)
file_writer = tf.summary.create_file_writer(scalarlogdir)
file_writer.set_as_default()

In [5]:
def lr_schedule(epoch):
    """Returns a custom learning rate that decreases as epochs progress.    """
    learning_rate = 0.02
    if epoch > 2:
        learning_rate = 0.002
    if epoch > 4:
        learning_rate = 0.0001
    if epoch > 6:
        learning_rate = 0.00001
    if epoch > 8:
        learning_rate = 0.00001

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [6]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
earlystops = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [8]:
callbacks = [tensorboard_callback,
             earlystops, lr_callback]

In [9]:
history = model.fit(x=train_x, y=train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(test_x, test_y),
                    shuffle=True,
                    callbacks=callbacks)

W0922 21:58:47.541370 18892 deprecation.py:323] From D:\anaconda\envs\tensorflow2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 135us/sample - loss: 1.1677 - accuracy: 0.6014 - val_loss: 0.2563 - val_accuracy: 0.9261
Epoch 2/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.4242 - accuracy: 0.8754 - val_loss: 0.1635 - val_accuracy: 0.9515
Epoch 3/10
60000/60000 [==============================] - 8s 127us/sample - loss: 0.3094 - accuracy: 0.9133 - val_loss: 0.1313 - val_accuracy: 0.9625
Epoch 4/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.2535 - accuracy: 0.9296 - val_loss: 0.1195 - val_accuracy: 0.9657
Epoch 5/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.2444 - accuracy: 0.9331 - val_loss: 0.1156 - val_accuracy: 0.9666
Epoch 6/10
60000/60000 [==============================] - 8s 137us/sample - loss: 0.2411 - accuracy: 0.9331 - val_loss: 0.1155 - val_accuracy: 0.9670
Epoch 7/10
60000/60000 [==========================

In [10]:
model.evaluate(test_x, test_y)

10000/10000 [==============================] - 1s 75us/sample - loss: 0.1157 - accuracy: 0.9672


[0.11570903295725583, 0.9672]

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840 

In [12]:
model.save('mymodel.h5')

In [13]:
model.save('mymodel')